In [5]:
# Import auxiliary libraries
import numpy as np
import math

# Import Qiskit
from qiskit import Aer
from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.utils import QuantumInstance, algorithm_globals

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

# Functions

In [6]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()
    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)
    
    return optimizer.solve(quadprog)

In [7]:
def get_quantum_solution_for(
    algo, quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count
    
    # Create solver and optimizer
    if algo == "QAOA":
        solver = QAOA(optimizer=optimizer, quantum_instance=quantumInstance, callback=callback,)
    if algo == "VQE":
        solver = VQE(optimizer=optimizer, quantum_instance=quantumInstance, callback=callback)

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

# QC / Simulator

In [8]:
backend = Aer.get_backend("qasm_simulator")
algorithm_globals.random_seed = 42
# Create a QuantumInstance
simulator_instance = QuantumInstance(
    backend=backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

# Program

In [9]:
nb_stick = 10
past = ["/", "¬", "¬", "x", "/", "¬", "¬", "/"]

# Check number of stick max
if nb_stick >= 3:
    max_stick = 3
else:
    max_stick = nb_stick

# Check the past
# |||||||| ## /¬/¬¬/
past.reverse()
poten_stick = nb_stick
for i in range(len(past)):
    if past[i] == "/":
        poten_stick += 0.5
    if past[i] == "¬":
        u = 1
        if len(past)-1 >= i+u:
            while past[i+u] == "¬":
                u += 1
            if past[i+u] == "/":
                poten_stick += 0.5

# Check last turn
last_st = 0
if past[0] == "¬":
    u = 1
    while past[0+u] == "¬":
        u += 1
        if past[0+u] == "/":
            last_st = 0.5
if past[0] == "/":
        last_st = 0.5
            
print("Nb_stick = ", nb_stick)
print("Poten_stick = ", poten_stick)
print("Max_stick = ", max_stick)
print("Last_st = ", last_st)

Nb_stick =  10
Poten_stick =  13.5
Max_stick =  3
Last_st =  0.5


In [10]:
quadprog = QuadraticProgram(name="qnim")
quadprog.integer_var(name="x", lowerbound=0, upperbound=max_stick)
quadprog.integer_var(name="sup", lowerbound=0, upperbound=max_stick)
quadprog.integer_var(name="intric", lowerbound=0, upperbound=max_stick)
quadprog.maximize(
    linear={"x":1, "sup":0.5, "intric":last_st},
    quadratic={("sup", "intric"):0.5}
)
# General constraints
quadprog.linear_constraint(linear={"x":1, "sup":1, "intric":1}, sense=">", rhs=0, name="gen_min")
quadprog.linear_constraint(linear={"x":1, "sup":1, "intric":1}, sense="<=", rhs=max_stick, name="gen_max")

#quadprog.quadratic_constraint(quadratic={("sup", "intric"):1, ("x", "sup"):1}, sense="<=", rhs=poten_stick%4-1, name=qvers)

# Mod4 constraints
if math.ceil((poten_stick-0.5)%4) > 0:
    quadprog.linear_constraint(linear={"x":1, "sup":1}, sense="<=", rhs=math.ceil((poten_stick-0.5)%4), name="qua_mod4")
if (nb_stick-1)%4 > 0:
    quadprog.linear_constraint(linear={"x":1, "sup":1, "intric":1}, sense="<=", rhs=(nb_stick-1)%4, name="cla_mod4")

print(quadprog.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qnim

Maximize
 obj: x + 0.500000000000 sup + 0.500000000000 intric + [ sup*intric ]/2
Subject To
 gen_min: x + sup + intric >= 0
 gen_max: x + sup + intric <= 3
 qua_mod4: x + sup <= 1
 cla_mod4: x + sup + intric <= 1

Bounds
       x <= 3
       sup <= 3
       intric <= 3

Generals
 x sup intric
End



In [11]:
# Estimate the number of qubits required
ising_operations, _ = (QuadraticProgramToQubo().convert(quadprog).to_ising())
print(f"Number of qubits required is {ising_operations.num_qubits}")

Number of qubits required is 14


# Solving

## Classical solution

In [12]:
# Get classical result
classical_result = get_classical_solution_for(quadprog)

# Format and print result
print("Solution found using the classical method:\n")
print(f"Maximum state is {classical_result.fval} tons")
print(f"Gates used are: ")

_actions = [v.name for v in quadprog.variables]
for actionGate, nbActions in enumerate(classical_result.x):
    print(f"\t{nbActions} actions of {_actions[actionGate]}")

Solution found using the classical method:

Maximum state is 1.0 actions
Gates used are: 
	1.0 actions of x
	0.0 actions of sup
	0.0 actions of intric


## QAOA

In [18]:
# Get QAOA result
final_result = []
qaoa_result, qaoa_eval_count = get_quantum_solution_for("QAOA", quadprog, simulator_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum state is {qaoa_result.fval} actions")
print(f"Gates used are: ")
for nbActions, actionGate in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{nbActions} actions of {actionGate}")
    for i in range(int(nbActions)):
        final_result.append(actionGate)

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

Solution found using the QAOA method:

Maximum state is 1.0 actions
Gates used are: 
	1.0 actions of x
	0.0 actions of sup
	0.0 actions of intric

The solution was found within 42 evaluations of QAOA.


In [19]:
print("Gates to use : ", final_result)

Gates to use :  ['x']
